In [ ]:
# ============================================================================
# 1. HÜCRE: KURULUM
# ============================================================================
print("Kütüphaneler kuruluyor...")
!pip install transformers accelerate bitsandbytes -q

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import textwrap
import os

# GPU Kontrolü
if torch.cuda.is_available():
    print(f"✅ GPU Algılandı: {torch.cuda.get_device_name(0)}")
else:
    print("❌ GPU bulunamadı! Lütfen A100 GPU seçtiğinizden emin olun.")

Kütüphaneler kuruluyor...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 40.8 MB/s eta 0:00:00
✅ GPU Algılandı: NVIDIA A100-SXM4-40GB


In [ ]:
# ============================================================================
# 2. HÜCRE: MODELİ YÜKLEME
# ============================================================================
# Doğru Model İsmi:
model_id = "Trendyol/Trendyol-LLM-7b-chat-dpo-v1.0"

print(f"🚀 '{model_id}' modeli yükleniyor... (Bu işlem 1-2 dakika sürebilir)")

try:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        load_in_8bit=False
    )

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device_map="auto"
    )
    print("✅ Model başarıyla yüklendi!")

except Exception as e:
    print(f"❌ HATA: {e}")

🚀 'Trendyol/Trendyol-LLM-7b-chat-dpo-v1.0' modeli yükleniyor... (Bu işlem 1-2 dakika sürebilir)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Model başarıyla yüklendi!


In [ ]:
# ============================================================================
# 3. HÜCRE: DERS NOTUNU OKUMA
# ============================================================================
dosya_adi = "İLİŞKİ TÜRLERİ.txt"
metin_icerigi = ""

if os.path.exists(dosya_adi):
    with open(dosya_adi, "r", encoding="utf-8") as f:
        metin_icerigi = f.read()
    print(f"📄 '{dosya_adi}' başarıyla okundu ({len(metin_icerigi)} karakter).")
else:
    print(f"❌ HATA: '{dosya_adi}' bulunamadı. Lütfen sol menüye yükleyin.")

❌ HATA: 'İLİŞKİ TÜRLERİ.txt' bulunamadı. Lütfen sol menüye yükleyin.


In [ ]:
# ============================================================================
# 4. HÜCRE: KLASİK SORU ÜRETME (ÖRNEKLİ)
# ============================================================================
def klasik_soru_uret_v3(metin_parcasi):

    # Modele "Bak bu örnekteki gibi yap" diyoruz.
    prompt = f"""Aşağıdaki metni oku ve metnin ana fikriyle ilgili, üniversite seviyesinde, akademik dille yazılmış bir Klasik (Açık Uçlu) sınav sorusu ve cevabını yaz.

ÖRNEK:
Metin: Bitcoin, merkeziyetsiz bir dijital para birimidir. İşlemler blokzincir adı verilen dağıtık bir defterde tutulur. Bu yapı, herhangi bir merkezi otoriteye ihtiyaç duymadan güvenliği sağlar.
Soru: Bitcoin'in merkeziyetsiz yapısı ne anlama gelir ve işlemler nerede kaydedilir?
Cevap: Bitcoin'in herhangi bir merkeze veya otoriteye bağlı olmadığı anlamına gelir. İşlemler blokzincir (blockchain) adı verilen dağıtık defterde şeffaf bir şekilde tutulur.

GÖREV:
Metin:
{metin_parcasi}

Soru:"""

    outputs = pipe(
        prompt,
        max_new_tokens=250,
        do_sample=True,
        temperature=0.1, # Düşük sıcaklık = Daha az saçmalama, daha çok örneğe sadakat
        top_p=0.9
    )

    # Sadece üretilen kısmı al
    generated = outputs[0]["generated_text"]

    # Prompt'tan sonrasını temizleyip al
    if "Soru:" in generated:
        return "Soru:" + generated.split("Soru:")[-1].strip()
    else:
        return generated.split(metin_parcasi)[-1].strip()

print("✅ Örnekli Klasik Soru fonksiyonu (v3) hazır.")

✅ Örnekli Klasik Soru fonksiyonu (v3) hazır.


In [ ]:
# ============================================================================
# 5. HÜCRE: ÇOKTAN SEÇMELİ SORU ÜRETME (ÖRNEKLİ)
# ============================================================================
def coktan_secmeli_uret_v3(metin_parcasi):

    prompt = f"""Aşağıdaki metni oku ve metne dayalı, 5 şıklı, akademik dille yazılmış zor bir çoktan seçmeli soru hazırla.

ÖRNEK:
Metin: Kriptografi, verilerin güvenliğini sağlamak için matematiksel yöntemler kullanır. Simetrik ve asimetrik olmak üzere ikiye ayrılır. Simetrik şifrelemede tek anahtar kullanılırken, asimetrikte bir çift anahtar vardır.
Soru: Aşağıdakilerden hangisi kriptografinin temel amaçlarından ve türlerinden biriyle ilgili doğru bir ifadedir?
A) Veri hızını artırmak için kullanılır.
B) Sadece tek bir şifreleme yöntemi vardır.
C) Asimetrik şifrelemede şifreleme ve çözme için farklı anahtarlar kullanılır.
D) Simetrik şifreleme daha güvenlidir.
E) Donanım maliyetini düşürmek temel amaçtır.
Doğru Cevap: C) Asimetrik şifrelemede şifreleme ve çözme için farklı anahtarlar kullanılır.
Açıklama: Metinde belirtildiği gibi asimetrik şifreleme anahtar çifti kullanır.

GÖREV:
Metin:
{metin_parcasi}

Soru:"""

    outputs = pipe(
        prompt,
        max_new_tokens=400,
        do_sample=True,
        temperature=0.1, # Düşük sıcaklık = Daha mantıklı şıklar
        top_p=0.9
    )

    generated = outputs[0]["generated_text"]

    if "Soru:" in generated:
        return "Soru:" + generated.split("Soru:")[-1].strip()
    else:
        return generated.split(metin_parcasi)[-1].strip()

print("✅ Örnekli Test Sorusu fonksiyonu (v3) hazır.")

✅ Örnekli Test Sorusu fonksiyonu (v3) hazır.


In [ ]:
# ============================================================================
# 6. HÜCRE: SORU ÜRETİMİNİ BAŞLAT
# ============================================================================
if metin_icerigi:
    print("🚀 Soru Üretimi Başlıyor (Örnekli Yöntem)...\n")

    # Metni mantıksal paragraflara böl
    chunks = metin_icerigi.split('\n\n')

    # Çok kısa parçaları ele
    valid_chunks = [c.strip() for c in chunks if len(c.strip()) > 100]

    print(f"Toplam {len(valid_chunks)} adet içerik bloğu işlenecek.\n")

    tum_sorular = ""

    for i, chunk in enumerate(valid_chunks):
        print(f"--- BLOK {i+1} İŞLENİYOR ---")

        # 1. Klasik Soru (v3)
        print("   > Klasik Soru...")
        try:
            klasik = klasik_soru_uret_v3(chunk)
        except:
            klasik = "Hata oluştu."

        # 2. Çoktan Seçmeli Soru (v3)
        print("   > Çoktan Seçmeli Soru...")
        try:
            test_sorusu = coktan_secmeli_uret_v3(chunk)
        except:
            test_sorusu = "Hata oluştu."

        # Ekrana Bas
        print("\n" + "*"*40)
        print(f"📝 BLOK {i+1} SONUÇLARI")
        print("*"*40)
        print(f"[KLASİK]\n{klasik}\n")
        print(f"[TEST]\n{test_sorusu}")
        print("\n" + "="*60 + "\n")

        # Dosyaya kaydetmek için biriktir
        tum_sorular += f"--- BLOK {i+1} ---\nMETİN:\n{chunk[:100]}...\n\n[KLASİK]\n{klasik}\n\n[TEST]\n{test_sorusu}\n\n{'='*50}\n\n"

    # Sonuçları Kaydet
    with open("uretilen_sorular_v3.txt", "w", encoding="utf-8") as f:
        f.write(tum_sorular)
    print("✅ Tüm sorular 'uretilen_sorular_v3.txt' dosyasına kaydedildi.")

In [ ]:
# ============================================================================
# 7. HÜCRE: API VE CLOUDFLARE TÜNELİ (FİNAL V7 - ARTIRILMIŞ KAPASİTE)
# ============================================================================
import os
import re
import time
import subprocess
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

# 1. Gerekli paketleri sessizce kur
print("⚙️ Gerekli kütüphaneler kontrol ediliyor...")
os.system("pip install fastapi uvicorn nest_asyncio pyngrok -q")

# 2. Cloudflare aracını indir (Eğer yoksa)
if not os.path.exists("cloudflared-linux-amd64"):
    print("☁️ Cloudflare tünel aracı indiriliyor...")
    os.system("wget -q -nc https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64")
    os.system("chmod +x cloudflared-linux-amd64")

# --- MODEL FONKSİYONLARI (Zaten Hafızada Olan 'pipe' kullanılır) ---

# 1. KLASİK SORU (Kavramsal Odaklı - 700 Token)
def guclu_klasik_soru_uret(metin_parcasi):
    # Metin çok uzunsa kırp
    if len(metin_parcasi) > 2000: metin_parcasi = metin_parcasi[:2000] + "..."

    prompt = f"""[INST] Sen uzman bir akademisyensin. Aşağıdaki ders notunu analiz et.
    GÖREV: Metindeki kavramları derinlemesine irdeleyen, "Neden, Nasıl, Arasındaki Fark Nedir" gibi düşündürücü BİR ADET Klasik sınav sorusu ve cevabı yaz.

    KURALLAR:
    1. Basit tanımlar sorma (Örn: "X nedir?" sorma).
    2. İlişkileri veya süreçleri sor.
    3. Asla metni özetleme.
    4. Cevabı detaylı yaz, yarıda kesme.

    METİN:
    === METİN BAŞLIYOR ===
    {metin_parcasi}
    === METİN BİTİYOR ===
    [/INST]
    Soru:"""

    # max_new_tokens=700 yaptık ki cevap yarıda kesilmesin
    outputs = pipe(prompt, max_new_tokens=700, do_sample=True, temperature=0.2, top_p=0.9)
    generated = outputs[0]["generated_text"]

    if "[/INST]" in generated: generated = generated.split("[/INST]")[-1].strip()
    return generated if generated.startswith("Soru:") else "Soru: " + generated

# 2. TEST SORUSU (Detay Odaklı - 400 Token)
def guclu_test_sorusu_uret(metin_parcasi):
    if len(metin_parcasi) > 2000: metin_parcasi = metin_parcasi[:2000] + "..."

    prompt = f"""[INST] Sen uzman bir sınav hazırlayıcısısın. Aşağıdaki ders notunu analiz et.
    GÖREV: Metindeki spesifik bir detaya, maddeye veya özelliğe odaklanan 5 şıklı zor bir test sorusu hazırla.

    İPUCU: "Aşağıdakilerden hangisi yanlıştır?", "Hangisi bu sürecin bir parçası değildir?" gibi ayırt edici sorular sor.

    METİN:
    === METİN BAŞLIYOR ===
    {metin_parcasi}
    === METİN BİTİYOR ===
    [/INST]
    Soru:"""

    outputs = pipe(prompt, max_new_tokens=400, do_sample=True, temperature=0.1, top_p=0.9)
    generated = outputs[0]["generated_text"]

    if "[/INST]" in generated: generated = generated.split("[/INST]")[-1].strip()
    return generated if generated.startswith("Soru:") else "Soru: " + generated

# 3. ÖZETLEME (Kapsamlı - 700 Token)
def guclu_ozetle(metin_parcasi):
    if len(metin_parcasi) > 2500: metin_parcasi = metin_parcasi[:2500] + "..."

    prompt = f"""[INST] Sen titiz bir akademik asistansın. Aşağıdaki ders notunu oku ve TÜM ÖNEMLİ NOKTALARI içeren kapsamlı bir özet çıkar.

    KURALLAR:
    1. Sadece tanım yapma; metinde geçen süreçleri, teknikleri ve faydaları da ekle.
    2. Teknik terimleri (Singleton, Polymorphism, UML vb.) olduğu gibi koru.
    3. Akıcı bir Türkçe kullan, maddeler halinde değil, dolgun paragraflar şeklinde yaz.
    4. Asla uydurma kelime kullanma.

    METİN:
    === METİN BAŞLIYOR ===
    {metin_parcasi}
    === METİN BİTİYOR ===

    GÖREV: Metnin tamamını kapsayan detaylı bir özet yaz.
    [/INST]
    ÖZET:"""

    # max_new_tokens=700 yaptık ki özet "tanıtım cümlesi" gibi kalmasın
    outputs = pipe(prompt, max_new_tokens=700, do_sample=True, temperature=0.2, top_p=0.9)
    generated = outputs[0]["generated_text"]

    if "[/INST]" in generated: generated = generated.split("[/INST]")[-1].strip()
    return generated.replace("ÖZET:", "").strip()

# --- API AYARLARI ---
class MetinIstegi(BaseModel):
    metin: str

app = FastAPI()

@app.post("/soru-uret")
async def soru_endpoint(istek: MetinIstegi):
    print(f"📥 Soru İsteği Geldi (Uzunluk: {len(istek.metin)})")
    try:
        klasik = guclu_klasik_soru_uret(istek.metin)
        test = guclu_test_sorusu_uret(istek.metin)
        return {"durum": "basarili", "klasik_soru": klasik, "test_sorusu": test}
    except Exception as e:
        return {"durum": "hata", "mesaj": str(e)}

@app.post("/ozet-uret")
async def ozet_endpoint(istek: MetinIstegi):
    print(f"📝 Özet İsteği Geldi (Uzunluk: {len(istek.metin)})")
    try:
        ozet = guclu_ozetle(istek.metin)
        return {"durum": "basarili", "ozet": ozet}
    except Exception as e:
        return {"durum": "hata", "mesaj": str(e)}

@app.get("/")
def ana_sayfa(): return {"mesaj": "Noto AI V7 (Full Kapasite) Çalışıyor! 🚀"}

# --- SUNUCUYU BAŞLATMA ---
nest_asyncio.apply()

# 1. Eski tünelleri temizle
subprocess.run(["pkill", "cloudflared"])
time.sleep(2)

# 2. Tüneli başlat ve çıktıyı dosyaya yaz
print("🔗 Cloudflare Tüneli başlatılıyor...")
logfile = open("cloudflared.log", "w")
subprocess.Popen(
    ["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8000"],
    stdout=logfile,
    stderr=logfile
)

# 3. Linkin oluşmasını bekle ve dosyadan oku
time.sleep(10) # Linkin oluşması için süre tanı
print("🔍 Link aranıyor...")

found_url = None
try:
    with open("cloudflared.log", "r") as f:
        log_content = f.read()
        match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', log_content)
        if match: found_url = match.group(0)
except: pass

if found_url:
    print("\n" + "="*60)
    print(f"🚀 BAŞARILI! Linkin: {found_url}")
    print("="*60 + "\n")
    print("👉 Bu linki PyCharm'daki 'ai_service.py' dosyasına yapıştır!")
else:
    print("\n⚠️ Link log dosyasından okunamadı. Lütfen sol menüden 'cloudflared.log' dosyasını açıp linki manuel kontrol et.\n")

# 4. Sunucuyu başlat
config = uvicorn.Config(app, port=8000)
server = uvicorn.Server(config)
await server.serve()

⚙️ Gerekli kütüphaneler kontrol ediliyor...
🔗 Cloudflare Tüneli başlatılıyor...
🔍 Link aranıyor...

🚀 BAŞARILI! Linkin: https://formatting-drill-pizza-outstanding.trycloudflare.com

👉 Bu linki PyCharm'daki 'ai_service.py' dosyasına yapıştır!


INFO:     Started server process [624]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
